<a href="https://colab.research.google.com/github/SaruniaOwO/KI/blob/main/rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -r requirements.txt


In [ ]:
!pip install langchain-huggingface
!pip install langchain-community
!pip install langchain huggingface_hub transformers

In [ ]:
!pip install langchain_openai

In [8]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('rag')

In [9]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model= "gpt-4o-mini")

In [ ]:
model.invoke("Was ist die Haupstadt von Deutschland?")

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Was ist die Haupstadt von Deutschland?")

In [14]:
from langchain.prompts import ChatPromptTemplate

template = """
Beantworte die Frage basierend auf dem Kontext.
Wenn Du die Frage nicht beantworten kannst, antworte "Ich weißt es nicht!".

context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Rakfs Bruder heißt Axl.", question="Wer ist Ralfs Bruder?")

'Human: \nBeantworte die Frage basierend auf dem Kontext.\nWenn Du die Frage nicht beantworten kannst, antworte "Ich weißt es nicht!".\n\ncontext: Rakfs Bruder heißt Axl.\n\nQuestion: Wer ist Ralfs Bruder?\n'

In [ ]:
chain = prompt | model | parser
chain.invoke({
    "context": "Rakfs Bruder heißt Axl.",
    "question": "Wer ist Ralfs Bruder?"
})

In [17]:
translation_prompt = ChatPromptTemplate.from_template("Übersetze {answer} in {language}")

In [19]:
from operator import itemgetter

translation_chain = (
    {"awnser": chain, "language": itemgetter("language")}  | translation_chain | model | parser
)

In [ ]:
translation_chain.invoke(
    {
        "context": "Rakfs Bruder heißt Axl. Er hat zwei weitere Geschwister.",
        "question": "Wie viele Geschwister hat Ralf?",
        "language": "Englisch"
    }
)

In [24]:
with open("interview.txt") as file:
  interview = file.read()

interview[:100]

"I think it's possible that physics has exploits and we should be trying to find them. arranging some"

In [ ]:
chain.invoke({
    "context": interview,
    "question": "Is reading papers a good idea?"
})

In [25]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("interview.txt")
documents = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_spltter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
text_spltter.split_documents(text_documents)[:5]
documents = text_spltter.split_documents(text_documents)


In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
emedded_query = embeddings.embed_query("Wer ist Ralfs Bruder")

print({lenemedded_query[:10]})
print(embedded_query[:10])

In [ ]:
sentence1 = embeddings.embed_query("Ralfs Bruder ist Axl")
sentence2 = embeddings.embed_query("Michaelas Schwester ist Barbara")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

query_sentence1_similarity = cosine_similarity([embedded_query] , [sentence1]) [0] [0]
query_sentence2_similarity = cosine_similarity([embedded_query] , [sentence2]) [0] [0]

query_sentence1_similarity
query_sentence2_similarity

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore1 = DocArrayInMemorySearch.from_texts(
    [
        "Ralfs Bruder ist Axl",
        "Michaelas Schwester ist Barbara",
        "Simon mag weiße Autos",
        "Sara hat zwei Geschwister",
        "Jannis Vater ist Anlagemechaniker",
        "Tassilo fährt einen Seat",
    ],
    embedings=embeddings
)

In [ ]:
vectorstore1.similarity_search_with_score(query="Wer ist Ralfs Bruder?", k=3)

In [ ]:
retriever1 = vectorstore1.as_retriever()
retriever1.invoke("Wer ist Ralfs Bruder?")

In [ ]:
from langchain_core.runnables import RunnableParalel, RunnablePassthrough

In [ ]:
vrctorstore2 = DocArrayInMemorySearch.from_documents(
    documents=documents,
    embedding=embeddings
)

chain = (
    {"context": vectorstore2.as_retriever(), "question": RunnablePassthrough} | prompt | model | parser
)

In [ ]:
chain.invoke("What is synthetic intelligence?")